In [23]:
import pandahouse as ph

import warnings
warnings.filterwarnings("ignore")

## SQL

**1. Очень усердные ученики EDA**


In [24]:
#подключаемся к clickhouse
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student',
                      'password':'dpo_python_2020'
                     }

In [25]:
#создаем запрос, выгружаем данные, чтобы провести EDA
q = '''
    SELECT
        st_id,
        toDateTime(timest) AS timest,
        correct,
        subject

    FROM
        {db}.peas
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
peas = ph.read_clickhouse(query=q, connection=connection_default)
peas.head()

,st_id,timest,correct,subject
0,100379,2021-10-30 13:32:29,1,Theory of probability
1,100379,2021-10-30 14:11:19,0,Vizualization
2,100379,2021-10-30 15:54:22,1,Theory of probability
3,100379,2021-10-30 16:44:50,1,Vizualization
4,100379,2021-10-30 17:15:05,1,Theory of probability


In [26]:
peas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13245 entries, 0 to 13244
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   st_id    13245 non-null  object        
 1   timest   13245 non-null  datetime64[ns]
 2   correct  13245 non-null  int8          
 3   subject  13245 non-null  object        
dtypes: datetime64[ns](1), int8(1), object(2)
memory usage: 323.5+ KB


In [27]:
peas.isna().sum()

st_id      0
timest     0
correct    0
subject    0
dtype: int64

In [28]:
peas.nunique()

st_id        308
timest     12238
correct        2
subject        5
dtype: int64

In [29]:
(peas.timest.min(), peas.timest.max())

(Timestamp('2021-10-30 13:00:00'), Timestamp('2021-10-31 17:29:52'))

Типы даннных ок, пропущенных значений нет, 308 уникальных пользователей, 2 значения в столбце correct, 5 учебных предметов, временной диапазон: с 13:00:00 30.10.21 по 17:29:52 31.10.21 - чуть больше суток.
****
На первый взгляд с данными все в порядке, приступаем к написанию запросов

### **1.2 Запрос**

Дана таблица default.peas. Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [30]:
q = '''
    SELECT COUNT(DISTINCT st_id) AS diligent_st

    FROM
    (
        SELECT st_id,
                toMonth(timest) as Month,
                SUM(correct) AS correct_peas
        FROM {db}.peas
        GROUP BY st_id, Month
        HAVING correct_peas >= 20
    )
    '''

task1 = ph.read_clickhouse(query=q, connection=connection_default)
task1

,diligent_st
0,136


In [31]:
print('Количество очень усердных студентов: ',task1.diligent_st.values[0])

Количество очень усердных студентов:  136


**2. Сводная таблица метрик**

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. *Команда провела эксперимент, где был протестирован новый экран оплаты.*

Необходимо **в одном запросе** выгрузить следующую информацию о группах пользователей:
- ARPU
- ARPAU
- CR в покупку
- СR активного пользователя в покупку
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
- ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [32]:
#выгружаем оставшиеся таблицы и проводим EDA
q2 = '''
    SELECT *

    FROM
        {db}.studs
    '''

studs = ph.read_clickhouse(query=q2, connection=connection_default)
studs.head()

,st_id,test_grp
0,100379,pilot
1,101432,control
2,104818,pilot
3,104885,pilot
4,104966,pilot


In [33]:
studs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   st_id     600 non-null    object
 1   test_grp  600 non-null    object
dtypes: object(2)
memory usage: 9.5+ KB


In [34]:
studs.isna().sum()

st_id       0
test_grp    0
dtype: int64

In [35]:
studs.nunique()

st_id       600
test_grp      2
dtype: int64

Все ок

In [36]:
q3 = '''
    SELECT
        st_id,
        toDateTime(sale_time) AS sale_time,
        money,
        subject

    FROM
        {db}.final_project_check
    '''

final_project_check = ph.read_clickhouse(query=q3, connection=connection_default)
final_project_check.head()

,st_id,sale_time,money,subject
0,101432,2021-10-31 04:44:32,85000,Math
1,101432,2021-10-31 12:43:50,65000,Vizualization
2,104885,2021-10-30 17:05:55,65000,Vizualization
3,104885,2021-10-30 22:49:33,75000,Statistics
4,106464,2021-10-31 13:17:13,85000,Math


In [37]:
final_project_check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   st_id      61 non-null     object        
 1   sale_time  61 non-null     datetime64[ns]
 2   money      61 non-null     uint32        
 3   subject    61 non-null     object        
dtypes: datetime64[ns](1), object(2), uint32(1)
memory usage: 1.8+ KB


In [38]:
final_project_check.isna().sum()

st_id        0
sale_time    0
money        0
subject      0
dtype: int64

Тоже все ок. Переходим к запросу

Этапы составления запроса:
1. Джойним все таблицы по st_id
2. Переносим джоин в подзапрос
3. Приступаем к расчету метрик
4. Понимаем что нужно сагрегировать данные в дф peas - агрегируем (подзапрос внутри подзапроса)
5. Расчитываем метрики и окргуляем их до 2 знаков после запятой
6. Группируем

In [39]:
q4 = '''
    SELECT studs.test_grp,
    ROUND(SUM(final.money)/COUNT(DISTINCT studs.st_id), 2) AS ARPU,
    ROUND(SUMIf(final.money, peas.activity > 10)/COUNTIf(DISTINCT studs.st_id, peas.activity > 10), 2) AS ARPAU,
    ROUND((COUNTIf(DISTINCT studs.st_id, final.money > 0)/COUNT(DISTINCT studs.st_id)) * 100, 2) AS CR,
    ROUND((COUNTIf(DISTINCT studs.st_id, final.money > 0 AND peas.activity > 10)
                            /COUNTIf(DISTINCT studs.st_id, peas.activity > 10)) * 100, 2) AS CR_active,
    ROUND((COUNTIf(DISTINCT studs.st_id, final.money > 0 AND peas.math_activity >= 2 and final.subject = 'Math')
                            /COUNTIf(DISTINCT studs.st_id, peas.math_activity >= 2)) * 100, 2) AS CR_math_active

    FROM
    (SELECT *
    FROM studs AS studs
    LEFT JOIN
         (SELECT st_id,
            SUM(correct) AS activity,
            SUMIf(correct, subject = 'Math') AS math_activity
        FROM peas
        GROUP BY st_id) AS peas
    ON studs.st_id = peas.st_id
    LEFT JOIN final_project_check AS final
    ON studs.st_id = final.st_id)

    GROUP BY studs.test_grp
    '''

task2= ph.read_clickhouse(query=q4, connection=connection_default)
task2

,studs.test_grp,ARPU,ARPAU,CR,CR_active,CR_math_active
0,control,4540.98,10393.70,4.92,11.02,6.12
1,pilot,11508.47,29739.58,10.85,26.04,9.52
